In [ ]:
# Unidade 4 – Aplicações com Python
# Atividade Prática
# Bruno Guimarães

# ---- Passo 1: Importar bibliotecas e carregar dados ----
import os
import random
import numpy as np
import tensorflow as tf
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

# (Opcional) fixar sementes para reprodutibilidade
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Carregar conjunto de dados Iris
iris = load_iris()
X = iris.data          # array (150, 4) com as 4 features
y = iris.target        # array (150,) com rótulos 0, 1, 2
class_names = iris.target_names  # ['setosa', 'versicolor', 'virginica']

# Transformar em DataFrame (opcional, só para visualização)
df = pd.DataFrame(X, columns=iris.feature_names)
df["target"] = y
print("Amostras iniciais:\n", df.head(), "\n")

# ---- Passo 2: Pré-processamento (split + normalização) ----
# Split treino/teste com estratificação para manter proporção das classes
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=SEED
)

# Normalizar features com StandardScaler (fit em treino; transform em treino e teste)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# ---- Passo 3: Construir o modelo (Keras Sequential) ----
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)),  # 4 features
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(8, activation="relu"),
    tf.keras.layers.Dense(3, activation="softmax")  # 3 classes
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",  # rótulos inteiros (0,1,2)
    metrics=["accuracy"]
)

model.summary()

# ---- Passo 4: Treinar o modelo ----
history = model.fit(
    X_train_scaled, y_train,
    epochs=80,            # pode ajustar (ex.: 50~150)
    batch_size=16,        # mini-batch
    validation_split=0.2, # separa parte do treino para validação
    verbose=0             # 0 = silencioso | 1 = barra | 2 = por época
)

print("Treinamento concluído.\n")

# ---- Passo 5: Avaliar o modelo no conjunto de teste ----
test_loss, test_acc = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Desempenho no teste: loss={test_loss:.4f} | accuracy={test_acc:.4f}\n")

# Métricas detalhadas
y_proba = model.predict(X_test_scaled, verbose=0)         # probabilidades
y_pred  = np.argmax(y_proba, axis=1)                      # classes previstas

print("Matriz de confusão:")
print(confusion_matrix(y_test, y_pred), "\n")

print("Classification report:")
print(classification_report(y_test, y_pred, target_names=class_names))

# ---- Passo 6: Fazer previsões (exemplos) ----
# Seleciona algumas amostras do X_test
num_show = min(5, len(X_test_scaled))
print(f"\n Algumas previsões ({num_show} amostras):")
for i in range(num_show):
    probs = y_proba[i]
    pred_class = y_pred[i]
    true_class = y_test[i]
    print(
        f"Ex{i+1}: true={class_names[true_class]:>10s} | "
        f"pred={class_names[pred_class]:>10s} | "
        f"probs={np.round(probs, 3)}"
    )

# (Opcional) salvar o modelo treinado
model.save("modelo_iris_tf.keras")
print("\n Modelo salvo em 'modelo_iris_tf.keras'.")


Amostras iniciais:
    sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0       0  
1       0  
2       0  
3       0  
4       0   



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 243 (972.00 B)

 Trainable params: 243 (972.00 B)

 Non-trainable params: 0 (0.00 B)

Treinamento concluído.

Desempenho no teste: loss=0.2917 | accuracy=0.8333

Matriz de confusão:
[[10  0  0]
 [ 0  5  5]
 [ 0  0 10]] 

Classification report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      0.50      0.67        10
   virginica       0.67      1.00      0.80        10

    accuracy                           0.83        30
   macro avg       0.89      0.83      0.82        30
weighted avg       0.89      0.83      0.82        30


Algumas previsões (5 amostras):
Ex1: true=    setosa | pred=    setosa | probs=[0.993 0.006 0.001]
Ex2: true= virginica | pred= virginica | probs=[0.005 0.252 0.743]
Ex3: true=versicolor | pred=versicolor | probs=[0.073 0.855 0.072]
Ex4: true=versicolor | pred=versicolor | probs=[0.035 0.906 0.059]
Ex5: true=    setosa | pred=    setosa | probs=[0.995 0.004 0.001]

Modelo salvo em 'modelo_iris_tf.keras'.
